# 选取rawcounts矩阵以及计算RPKM进行筛选
https://github.com/reneshbedre/bioinfokit

!pip install bioinfokit

!pip install openpyxl

In [56]:
import pandas as pd
from bioinfokit.analys import norm

In [65]:
fc_file = '../featureCounts/all_feature.txt'
df = pd.read_csv(fc_file,header=1,sep='\t')
df_geneid = df.iloc[:,0:1]
df_counts = df.iloc[:,5:]
df_merge = pd.concat([df_geneid,df_counts],axis=1)
df_merge.head(2)

,Geneid,Length,../bam/KD_NC-rep1_Aligned_sort_rmdup.bam,../bam/KD_NC-rep2_Aligned_sort_rmdup.bam,../bam/KD_NC-rep3_Aligned_sort_rmdup.bam,../bam/KD_CCNB1-rep1_Aligned_sort_rmdup.bam,../bam/KD_CCNB1-rep2_Aligned_sort_rmdup.bam,../bam/KD_CCNB1-rep3_Aligned_sort_rmdup.bam,../bam/KD_PUS7-rep1_Aligned_sort_rmdup.bam,../bam/KD_PUS7-rep2_Aligned_sort_rmdup.bam,../bam/KD_PUS7-rep3_Aligned_sort_rmdup.bam
0,DDX11L1,1652,0,0,1,0,0,0,0,0,0
1,WASH7P,1769,602,520,415,436,402,464,404,442,466


In [66]:
colName = ['Geneid','Length','NC1','NC2','NC3','CCNB11','CCNB12','CCNB13','PUS71','PUS72', 'PUS73']

In [67]:
# df = df.copy()
df_merge.columns=colName
df_merge.set_index('Geneid', inplace=True)
df_merge.head(2)

,Length,NC1,NC2,NC3,CCNB11,CCNB12,CCNB13,PUS71,PUS72,PUS73
Geneid,,,,,,,,,,
DDX11L1,1652,0,0,1,0,0,0,0,0,0
WASH7P,1769,602,520,415,436,402,464,404,442,466


In [68]:
# now, normalize raw counts using RPKM method
# gene length must be in bp
nm = norm()
nm.rpkm(df=df_merge, gl='Length')
# get RPKM normalized dataframe
df_rpkm = nm.rpkm_norm
df_rpkm.head(2)

,NC1,NC2,NC3,CCNB11,CCNB12,CCNB13,PUS71,PUS72,PUS73
Geneid,,,,,,,,,
DDX11L1,0.000000,0.000000,0.055467,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
WASH7P,24.162154,21.519855,21.496181,19.219697,20.008151,18.818344,23.000008,22.83763,23.241676


In [69]:
# save rpkm
df_rpkm.to_excel('plots/rpkm.xlsx')
# save raw_counts
df_merge.to_excel('plots/raw_counts.xlsx')

In [75]:
# 按照RPKM control组大于1的条件过滤基因
flt = df_rpkm.apply(lambda x: (x['NC1']>=1) and (x['NC2']>=1) and x['NC3']>=1, axis=1)

In [78]:
df_flt_rawcounts = df_merge[flt].copy()
df_flt_rawcounts.reset_index(inplace=True)
df_flt_rawcounts

,Geneid,Length,NC1,NC2,NC3,CCNB11,CCNB12,CCNB13,PUS71,PUS72,PUS73
0,WASH7P,1769,602,520,415,436,402,464,404,442,466
1,LOC729737,5474,456,362,335,411,372,433,372,432,462
2,MIR12136,142,1423,1778,2292,3635,2954,1319,3055,3109,3044
3,LOC100288069,7117,472,539,399,416,335,441,384,437,467
4,LINC00115,1317,27,25,22,25,20,23,29,20,21
...,...,...,...,...,...,...,...,...,...,...,...
11925,MAPK8IP2,5698,243,249,173,612,561,760,410,483,461
11926,ARSA,4294,218,188,161,247,257,302,242,246,244
11927,SHANK3,15337,307,330,268,680,530,732,332,328,393
11928,RPL23AP82,1864,125,117,94,168,139,182,120,145,145


In [83]:
df_flt_rawcounts.drop('Length', axis=1, inplace=True)

In [84]:
# 过滤后的raw counts进入DESeq2处理
df_flt_rawcounts.to_csv(fc_file.replace('.txt','_fix.csv'),index=None,sep='\t')